# GLTF 格式教學 工具包

In [0]:
class glTF_tools(object):
    def __init__(self):
        return


In [0]:
@staticmethod
def glb_loader(glb_bytes):
    import json
    assert glb_bytes[0:4].decode() == "glTF"

    gltf_length = 0

    for idx, curr in enumerate(glb_bytes[8:12]):
        gltf_length += curr * 256 ** idx

    assert len(glb_bytes) == gltf_length

    def chunk_parser(chunk_type, chunk_data):
        return {
            "BIN\u0000": lambda:chunk_data,
            "JSON": lambda:json.loads(chunk_data.decode()),
        }[chunk_type]()

    chunks = []
    offset = 12
    i = 0

    while(offset < len(glb_bytes)):
        chunks.append({ "type": "", "length": 0, "data": "" })

        for idx, curr in enumerate(glb_bytes[offset:offset + 4]):
            chunks[i]["length"] += curr * 256 ** idx

        chunks[i]["type"] = glb_bytes[offset + 4:offset + 8].decode()

        chunks[i]["data"] = chunk_parser(
            chunks[i]["type"],
            glb_bytes[offset + 8:offset + 8 + chunks[i]["length"]]
        )
        offset += 8 + chunks[i]["length"]
        i += 1

    model = chunks[0]["data"]
    buffers = []
    for idx in range(len(model['buffers'])):
        buffers.append(chunks[idx + 1]["data"])

    return model, buffers

glTF_tools.glb_loader=glb_loader

del glb_loader

In [0]:
@staticmethod
def render_JSON(json_data):
    import json
    import uuid
    from IPython.display import display_javascript, display_html

    json_str = ""
    if isinstance(json_data, dict):
        json_str = json.dumps(json_data)
    else:
        json_str = json_data
    _uuid = str(uuid.uuid4())

    display_html(
        """
        <div id="{_uuid}" style="width:100%;"></div>
        <script src="https://rawgit.com/caldwell/renderjson/master/renderjson.js"></script>
        <script>
            document.getElementById('{_uuid}').appendChild(renderjson({json_str}))
        </script>
        """.format(_uuid=_uuid, json_str=json_str),
        raw=True
    )

glTF_tools.render_JSON = render_JSON

del render_JSON

In [0]:
@staticmethod
def accessor_with_buffer(idx, model, buffers):
    _accessor = model['accessors'][idx]
    _buffer_view = model['bufferViews'][_accessor['bufferView']]
    _buffer = buffers[_buffer_view['buffer']]
    byteLength = _buffer_view['byteLength']
    byteOffset = _buffer_view['byteOffset']
    
    return _accessor, _buffer[byteOffset:byteOffset + byteLength]

glTF_tools.accessor_with_buffer = accessor_with_buffer

del accessor_with_buffer

In [0]:
@staticmethod
def accessor_norm(accessor):
    import copy
    
    _accessor = copy.deepcopy(accessor)

    # 做一些簡單的檢查，以確認必要的屬性是否存在
    if _accessor.get("bufferView") == None:
        raise AttributeError("The accessor does not have the attribute of bufferView")
    if _accessor.get("componentType") == None:
        raise AttributeError("The accessor does not have the attribute of componentType")
    if _accessor.get("count") == None:
        raise AttributeError("The accessor does not have the attribute of count")
    if _accessor.get("type") == None:
        raise AttributeError("The accessor does not have the attribute of type")

    buffer_offset = _accessor.get("bufferOffset")
    _accessor["bufferOffset"] = buffer_offset if buffer_offset != None else 0

    return _accessor
    
glTF_tools.accessor_norm = accessor_norm

del accessor_norm

In [0]:
@staticmethod
def buffer_view_norm(buffer_view, target = 34962):
    import copy

    _buffer_view = copy.deepcopy(buffer_view)

    # 做一些簡單的檢查，以確認必要的屬性是否存在
    if _buffer_view.get("buffer") == None:
        raise AttributeError("The bufferView does not have the attribute of buffer")
    if _buffer_view.get("byteLength") == None:
        raise AttributeError("The bufferView does not have the attribute of byteLength")
    if _buffer_view.get("byteOffset") == None:
        raise AttributeError("The bufferView does not have the attribute of byteOffset")

    byte_stride = _buffer_view.get("byteStride")
    _buffer_view["byteStride"] = byte_stride if byte_stride != None else 0

    _target = _buffer_view.get("target")
    _buffer_view["target"] = _target if _target != None else target

    return _buffer_view
    
glTF_tools.buffer_view_norm = buffer_view_norm

del buffer_view_norm

In [0]:
@staticmethod
def model_norm(model):
    import copy

    _model = copy.deepcopy(model)

    for idx, accessor in enumerate(_model['accessors']):
        _model['accessors'][idx] = glTF_tools.accessor_norm(accessor)

    for mesh in _model['meshes']:
        for primitive in mesh['primitives']:
            accessor = _model['accessors'][primitive.get('indices')]
            _model['bufferViews'][accessor.get('bufferView')] = glTF_tools.buffer_view_norm(_model['bufferViews'][accessor.get('bufferView')], 34963)

    for idx, buffer_view in enumerate(_model['bufferViews']):
        _model['bufferViews'][idx] = glTF_tools.buffer_view_norm(buffer_view)

    return _model
    
glTF_tools.model_norm = model_norm

del model_norm

In [0]:
# !wget https://github.com/CSP-GD/notes/raw/master/practice/file_format/gltf%E6%A0%BC%E5%BC%8F%E8%A7%A3%E6%9E%90/glb/cube.glb

# glb_file = open('./cube.glb', 'rb')

# glb_bytes = glb_file.read()

In [0]:
# model, buffers = glTF_tools.glb_loader(glb_bytes)

# glTF_tools.render_JSON(model)
# print(buffers)

In [0]:
# _model = glTF_tools.model_norm(model)

# glTF_tools.render_JSON(_model)

In [37]:
print("glTF_tools is loaded")

glTF_tools is loaded
